In [1]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import Libraries

# Warning
import warnings
warnings.filterwarnings("ignore")

# Main
import os
import numpy as np
import cv2

# Model
import tensorflow as tf
from keras.applications import VGG16
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Rescaling, Input
from keras.optimizers import Adam, Adamax
from keras.metrics import categorical_crossentropy
from keras.models import Model
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

In [12]:
PATH_ROOT = "/content/drive/MyDrive/colab_code/"  # Root for all folders
DATASET_MEMMAP_PATH = "/content/drive/MyDrive/dataset_synth/data.dat"
LABEL_NPY_PATH = "/content/drive/MyDrive/dataset_synth/label.npy"

VAL_DATASET_MEMMAP_PATH = "/content/drive/MyDrive/dataset_synth/test/data.dat"
VAL_LABEL_NPY_PATH = "/content/drive/MyDrive/dataset_synth/test/label.npy"

EPOCH_NUMBER = 30
BATCH_SIZE = 30
LEARNING_RATE = 1e-4

N_IMAGES_PER_FOLDER = 200
N_FOLDERS = 27
IMAGE_TOTAL_NUMBER = N_IMAGES_PER_FOLDER * N_FOLDERS
IMAGE_SIZE = 200
IMAGE_CHANNEL = 3
NUMBER_OF_CLASSES = 27

VAL_N_IMAGES_PER_FOLDER = 50
VAL_IMAGE_TOTAL_NUMBER = VAL_N_IMAGES_PER_FOLDER * N_FOLDERS

In [4]:
def load_data(path: str, shape: tuple) -> np.ndarray:
    params = {
        "filename": path,
        "dtype": "uint8",
        "mode": "r",
        "shape": shape
    }
    return np.memmap(**params)

In [5]:
data = load_data(DATASET_MEMMAP_PATH, shape=(IMAGE_TOTAL_NUMBER, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL))
label_raw = np.load(LABEL_NPY_PATH)
label = np.zeros((IMAGE_TOTAL_NUMBER, NUMBER_OF_CLASSES), dtype="int8")
label[np.arange(IMAGE_TOTAL_NUMBER), label_raw] = 1

In [6]:
trgen = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=.2,
    height_shift_range=.2,
    zoom_range=.2
)

train_dataset = trgen.flow(x=data, y=label, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
data_test = load_data(VAL_DATASET_MEMMAP_PATH, shape=(VAL_IMAGE_TOTAL_NUMBER, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL))
label_test_raw = np.load(VAL_LABEL_NPY_PATH)
label_test = np.zeros((VAL_IMAGE_TOTAL_NUMBER, NUMBER_OF_CLASSES), dtype="int8")
label_test[np.arange(VAL_IMAGE_TOTAL_NUMBER), label_test_raw] = 1

In [8]:
valgen = ImageDataGenerator()

test_dataset = valgen.flow(x=data_test, y=label_test, shuffle=False, batch_size=BATCH_SIZE)

In [13]:
inp = Input(shape=[IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL])
scaled_input = Rescaling(scale=1/255, offset=0)(inp)

base_model =tf.keras.applications.efficientnet.EfficientNetB3(
    include_top=False, weights="imagenet", input_tensor=scaled_input, pooling='max')

base_model.trainable = True

x = base_model.output
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
x = Dense(
    256,
    kernel_regularizer=regularizers.l2(l=0.016),
    activity_regularizer=regularizers.l1(0.006),
    bias_regularizer=regularizers.l1(0.006),
    activation='relu'
)(x)
x = Dropout(rate=.4, seed=123)(x)
output = Dense(NUMBER_OF_CLASSES, activation='softmax')(x)

model = Model(inputs=inp, outputs=output)

In [14]:
model.compile(
    Adamax(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
checkpoint = ModelCheckpoint(
    os.path.join(PATH_ROOT, "best_weights.h5"),
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [15]:
# Train the Model
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCH_NUMBER,
    callbacks=[checkpoint]
)

Epoch 1/30
180/180 [==============================] - 141s 511ms/step - loss: 11.6461 - accuracy: 0.0689 - val_loss: 20.0956 - val_accuracy: 0.0370
Epoch 2/30
180/180 [==============================] - 90s 496ms/step - loss: 10.3115 - accuracy: 0.2119 - val_loss: 12.2842 - val_accuracy: 0.1519
Epoch 3/30
180/180 [==============================] - 90s 499ms/step - loss: 9.0664 - accuracy: 0.4674 - val_loss: 9.0297 - val_accuracy: 0.5207
Epoch 4/30
180/180 [==============================] - 92s 508ms/step - loss: 8.1359 - accuracy: 0.6844 - val_loss: 8.1767 - val_accuracy: 0.6741
Epoch 5/30
180/180 [==============================] - 89s 490ms/step - loss: 7.4807 - accuracy: 0.8163 - val_loss: 8.8896 - val_accuracy: 0.5089
Epoch 6/30
180/180 [==============================] - 89s 493ms/step - loss: 7.0094 - accuracy: 0.8813 - val_loss: 6.6108 - val_accuracy: 0.9630
Epoch 7/30
180/180 [==============================] - 86s 474ms/step - loss: 6.5853 - accuracy: 0.9187 - val_loss: 7.0083 - v